In [7]:
# Continuous Bag of Words Model

# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

In [8]:
# Read the data

filename = "/Users/rgparekh/Documents/Personal/Rajesh/Data/text8.zip"

def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data
  
words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


In [10]:
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]

  # collections.Counter calculates the word frequency for each word
  # results are ordered in descending order of frequency
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))

  # dictionary of word indexes (starting with 0 for UNK)
  # UNK: 0, etc.
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
    
  # list of word indexes of the same length as words
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count

  # dictionary of index : word
  # 0: UNK etc.
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:15])
print('Sample data', data[:25])
for idx in data[:25]:
    print('Dictionary:', reverse_dictionary[idx])
del words  # Hint to reduce memory.


Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430), ('two', 192644), ('is', 183153), ('as', 131815), ('eight', 125285), ('for', 118445)]
Sample data [5241, 3084, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10593, 134, 1, 27747, 2, 1, 103, 855, 3, 1, 15106, 0]
Dictionary: anarchism
Dictionary: originated
Dictionary: as
Dictionary: a
Dictionary: term
Dictionary: of
Dictionary: abuse
Dictionary: first
Dictionary: used
Dictionary: against
Dictionary: early
Dictionary: working
Dictionary: class
Dictionary: radicals
Dictionary: including
Dictionary: the
Dictionary: diggers
Dictionary: of
Dictionary: the
Dictionary: english
Dictionary: revolution
Dictionary: and
Dictionary: the
Dictionary: sans
Dictionary: UNK


In [24]:
# Function to generate training data
# Idea: Generate a buffer (dequeue) from the data. Use center of the buffer as the target.
# The batch will include the center of the buffer as the word and randomly chosen words from
# the skip_window on either side as labels

# e.g., Data: ['Soccer', 'is', 'extremely', 'popular', 'in', 'Europe', 'and', 'Latin', 'America']
# num_skips = 4 and skip_window = 2
# buffer size will be 5. buffer = ['Soccer', 'is', 'extremely', 'popular', 'in']

# Essentially, the lable is the center word in the window
# batch: [['Soccer', 'is', 'popular', 'in'], [is', 'extremely', 'in', 'Europe'], ...]
# labels: ['extremely', 'popular', ...]

data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  batch = np.ndarray(shape=(batch_size, span-1), dtype=np.int32)
  # print('batch = ', batch)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32) #2D array batch_size X 1
  # print('labels = ', labels)
  buffer = collections.deque(maxlen=span)
  # print('buffer: ', buffer)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # print('buffer: ', buffer)
  # print('data_index: ', data_index)
  # print(batch_size // num_skips)
  for i in range(batch_size):
    # just for testing
    buffer_list = list(buffer)
    labels[i, 0] = buffer_list.pop(skip_window)
    batch[i] = buffer_list
    # iterate to the next buffer
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [di for di in data[:32]])
print('r data:', [reverse_dictionary[di] for di in data[:32]])


for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    for bi in batch:
        print('    batch:', [reverse_dictionary[ei] for ei in bi])
    # print('    batch:', [reverse_dictionary[ei] for ei in (for bi in batch)])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])
    

data: [5241, 3084, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10593, 134, 1, 27747, 2, 1, 103, 855, 3, 1, 15106, 0, 2, 1, 151, 855, 3585, 1, 195]
r data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'UNK', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term']

with num_skips = 2 and skip_window = 1:
    batch: ['anarchism', 'as']
    batch: ['originated', 'a']
    batch: ['as', 'term']
    batch: ['a', 'of']
    batch: ['term', 'abuse']
    batch: ['of', 'first']
    batch: ['abuse', 'used']
    batch: ['first', 'against']
    labels: ['originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used']

with num_skips = 4 and skip_window = 2:
    batch: ['anarchism', 'originated', 'a', 'term']
    batch: ['originated', 'as', 'term', 'of']
    batch: ['as', 'a', 'of', 'abuse']
    batch: ['a', 'term', 

In [25]:
# Training

batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 2 # How many words to consider left and right.
num_skips = 4 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 128 # Random set of words to evaluate similarity on.
valid_window = 500 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 128 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(), tf.device('/cpu:0'):

  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size,skip_window*2])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  # Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  softmax_weights = tf.Variable(
    tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset)
  # Compute the softmax loss, using a sample of the negative labels each time.
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=tf.reduce_sum(embed, 1),
                               labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))


# Iterations
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 5000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 20000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 8.348818
Nearest to right: throw, bogus, cantos, pathogens, literal, okavango, designation, cue,
Nearest to last: bernadette, osce, colon, colder, worship, logistical, habibullah, assists,
Nearest to place: insisted, scientific, eelam, bcl, heineken, prompting, bolden, gssp,
Nearest to about: vehicular, pastiches, tribunals, colons, meleon, commemoration, via, dozenal,
Nearest to home: church, codebase, reddy, crazy, laurentiis, wallabies, subsurface, reshaping,
Nearest to d: pitt, reason, kambojas, dependable, collide, countered, interrelated, ayya,
Nearest to within: letsie, innings, rupture, learner, glycosides, ez, envoy, ba,
Nearest to days: dioscorus, palin, bankhead, gtpase, florid, obey, induced, estuaries,
Nearest to level: diedrich, frames, kirtland, favorable, antipopes, hippodrome, impart, auto,
Nearest to site: arexx, bypassing, clavichord, ibadi, caveats, activation, withdrawals, smalltalk,
Nearest to i: hermit, mackey, rents, passwords

Nearest to white: oecs, brake, valium, fidelity, loud, recant, propellant, yorkshire,
Nearest to age: reductionism, emulators, donated, cuthbert, advisors, smoot, topalov, ssc,
Nearest to country: squandered, greta, trouser, cincinnati, decrypted, janice, roundhouse, superscription,
Nearest to important: relevance, fluid, illinois, nz, dft, directors, hemiparesis, castle,
Nearest to is: sprinkled, mated, recommending, presocratic, dup, questions, lsch, sufferings,
Nearest to e: immunization, skirts, wtc, sous, poee, marcel, hei, patriot,
Nearest to systems: prophylactic, unwittingly, erd, undesirable, headers, iarc, bayreuth, kryptonian,
Nearest to used: follett, pairs, earnings, illuminatus, spaniards, si, novello, sling,
Nearest to established: ifr, participatory, supers, apostates, griffey, gerrymandering, eesti, surveillance,
Nearest to kingdom: warriors, milton, evatt, housed, gonzalo, taxonomy, strut, halicarnassus,
Nearest to after: gc, aalen, zane, barks, symptomatic, rossby, h

Nearest to kingdom: states, nations, revitalized, puppies, blistering, redmond, fright, muharram,
Nearest to after: before, during, while, facilitated, following, pronounces, for, toll,
Nearest to there: they, these, longer, it, sagittarius, considered, maffei, ramjet,
Nearest to living: centerpiece, marlow, essences, surrenders, avignon, hitch, deuce, malthusian,
Nearest to c: dharma, affidavit, rollins, hayes, almaty, infamously, g, mileage,
Nearest to western: strategically, spontaneous, lasting, southern, ullman, structuralism, commentators, mamluk,
Nearest to most: best, more, some, turbine, draper, gra, among, exiles,
Nearest to into: through, between, from, eels, around, sheepdog, herero, doses,
Nearest to their: its, his, her, the, our, cgi, khagan, them,
Nearest to present: mixer, starbuck, mcintyre, guayaquil, unsafe, tribunals, andree, marcomanni,
Nearest to written: published, used, released, taken, given, tabular, created, common,
Nearest to law: brahmi, harbors, carnation

Nearest to e: am, warehouses, hofstadter, thrash, pressures, adventurers, j, interdisciplinarity,
Nearest to systems: system, programs, devices, ely, controlling, ritually, operations, solutions,
Nearest to used: seen, referred, written, described, available, designed, intended, given,
Nearest to established: founded, formed, developed, presented, adopted, discovered, started, written,
Nearest to kingdom: states, nations, evagrius, superconductivity, warriors, blistering, responds, redmond,
Nearest to after: before, during, while, since, through, despite, when, masamune,
Nearest to there: considered, broadband, longer, she, yes, camus, called, they,
Nearest to living: fired, hitch, centerpiece, avignon, marlow, exhorts, disbelief, droppings,
Nearest to c: cg, stahl, r, lamar, m, cunt, kidnapped, nine,
Nearest to western: eastern, southern, threes, gilt, strategically, american, northern, mamluk,
Nearest to most: more, many, some, less, best, draper, highly, exiles,
Nearest to into: thr

Nearest to to: will, would, could, must, might, cannot, should, may,
Nearest to that: which, what, where, however, hopeless, whose, when, who,
Nearest to created: developed, produced, formed, introduced, followed, built, established, discovered,
Nearest to thus: therefore, then, martyn, baffled, before, justifying, purposefully, sores,
Nearest to modern: traditional, medieval, classical, recent, common, many, meld, unknown,
Nearest to led: caused, returned, lead, contributed, seems, become, due, according,
Average loss at step 65000: 8.631724
Average loss at step 70000: 8.616992
Average loss at step 75000: 8.520124
Average loss at step 80000: 8.566887
Nearest to right: left, side, hand, aspirations, hajj, codebreaking, mocking, frac,
Nearest to last: next, first, final, previous, same, original, third, following,
Nearest to place: advantage, snout, order, places, auspices, victory, reciting, particular,
Nearest to about: approximately, concerning, over, wenzel, related, dozenal, hawke,

Average loss at step 85000: 8.539151
Average loss at step 90000: 8.439977
Average loss at step 95000: 8.440452
Average loss at step 100000: 7.575906
Nearest to right: left, side, aspirations, top, tutors, alumnus, mocking, behind,
Nearest to last: next, first, final, previous, earliest, following, third, fifth,
Nearest to place: advantage, snout, care, order, fulfillment, reciting, auspices, machina,
Nearest to about: concerning, wenzel, regarding, entitled, plus, asa, featureless, across,
Nearest to home: irving, mouth, protectorates, lex, nene, head, burned, kuwait,
Nearest to d: b, cartoonist, dumped, ecclesiastic, mysticism, diverging, andersen, citt,
Nearest to within: outside, in, throughout, through, into, across, between, masamune,
Nearest to days: months, weeks, years, hours, decades, minutes, sith, month,
Nearest to level: levels, performance, efficiency, elevation, speed, risk, cost, license,
Nearest to site: website, sites, page, pages, museum, com, section, udma,
Nearest t